In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import biosignalsnotebooks as bsnb
import wfdb
import numpy as np2

In [2]:
# METHODS AND UTILITIES

def electrocardiomatrix(distance, r_peaks, filtered_ecg, init_window, peaks_window):
    init_seg = int(0.2 * distance)
    fin_seg = int(1.3 * distance)
    all_segments = []
    for peak in r_peaks[init_window:init_window + peaks_window]:
        if peak - init_seg < 0:
            segment = filtered_ecg[0:peak + fin_seg]
        else:
            segment = filtered_ecg[peak - init_seg:peak + fin_seg]
        all_segments.append(segment[:,np.newaxis])
    if all_segments[0].shape[0] < all_segments[1].shape[0]:
        zeros = np.zeros(int(all_segments[1].shape[0])-int(all_segments[0].shape[0]))[:, np.newaxis]
        new_segment = np.concatenate((zeros, all_segments[0]))
        all_segments[0] = new_segment
    # If it is trying to obtain the last image from the last peak and it doesn't has enough samples, we just skip to the next user.
    try: 
        ecm = np.concatenate(all_segments, 1)
    except ValueError:
        return None

    return ecm.T

def normalize(signal):
    a, b = -1, 1
    c = b - a
    aux = (signal - np.min(signal)) / (np.max(signal) - np.min(signal))
    norm_ecg = c * aux + a
    return norm_ecg

def process_ecg(unfiltered_ecg, fs):
    # Step 1 of Pan-Tompkins Algorithm - ECG Filtering (Bandpass between 5 and 15 Hz)
    filtered_signal = bsnb.detect._ecg_band_pass_filter(unfiltered_ecg, fs)

    # Step 2 of Pan-Tompkins Algorithm - ECG Differentiation
    differentiated_signal = np.diff(filtered_signal)

    # Step 3 of Pan-Tompkins Algorithm - ECG Rectification
    squared_signal = differentiated_signal * differentiated_signal

    # Step 4 of Pan-Tompkins Algorithm - ECG Integration ( Moving window integration )
    nbr_sampls_int_wind = int(0.080 * fs)
    integrated_signal = np.zeros_like(squared_signal)
    cumulative_sum = squared_signal.cumsum()
    integrated_signal[nbr_sampls_int_wind:] = (cumulative_sum[nbr_sampls_int_wind:] - cumulative_sum[
                                                                                      :-nbr_sampls_int_wind]) / nbr_sampls_int_wind
    integrated_signal[:nbr_sampls_int_wind] = cumulative_sum[:nbr_sampls_int_wind] / np.arange(1, nbr_sampls_int_wind + 1)

    # Initialisation of the R peak detection algorithm
    rr_buffer, signal_peak_1, noise_peak_1, threshold = bsnb.detect._buffer_ini(integrated_signal, fs)

    # Detection of possible and probable R peaks
    probable_peaks, possible_peaks = bsnb.detect._detects_peaks(integrated_signal, fs)

    # Identification of definitive R peaks
    definitive_peaks = bsnb.detect._checkup(probable_peaks, integrated_signal, fs, rr_buffer, signal_peak_1,
                                            noise_peak_1, threshold)
    # Conversion to integer type.
    definitive_peaks = np.array(list(map(int, definitive_peaks)))

    # Correcting step
    map_integers = definitive_peaks - 40 * (fs / 1000)
    definitive_peaks_reph = np.array(list(map(int, map_integers)))
    return definitive_peaks_reph, filtered_signal

def save_ecm(norm_ecm, train_filled, test_filled, train_ecms, test_ecms, path_test, path_train, key, i, f, j):
    a = np.random.uniform(0, 1)
    if a <= 0.2 or train_filled:
        plt.savefig(path_test + '5peaks-heatmap-' + key + '-' + str(i))
        j += 1
        if j > test_ecms:
            filled_test = True

    elif a > 0.2 or test_filled:
        plt.savefig(path_train + '5peaks-heatmap-' + key + '-' + str(i))
        f += 1
        if f >= train_ecms:
            train_filled = True
    plt.close()
    plt.close('all')

In [5]:
fs = 1000
dat_file = '19140\\19140.dat'
ddbb_record = dat_file.replace(".dat", "")

sig, fields = wfdb.rdsamp(ddbb_record)
unfiltered_ecg = sig[:, 0]
peaks, filtered_ecg = process_ecg(unfiltered_ecg, fs)

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\AlejandroGuerrero\\Desktop\\ELEKTRA\\ELEKTRA-approach\\19140\\19140.hea'